In [8]:
import numpy as np
import pandas as pd
from collections import Counter
from sklearn.model_selection import train_test_split


In [9]:
class DecisionTreeNode:
    def __init__(self):
        self.feature_index = None
        self.threshold = None
        self.left = None
        self.right = None
        self.value = None

class CustomDecisionTree:
    def __init__(self, max_depth=None, max_features=None):
        self.max_depth = max_depth
        self.max_features = max_features
        self.root = None

    def fit(self, X, y):
        self.n_classes = len(np.unique(y))
        self.n_features = X.shape[1]
        self.root = self._grow_tree(X, y)

    def _grow_tree(self, X, y, depth=0):
        n_samples, n_features = X.shape
        node = DecisionTreeNode()

        # Stopping criteria
        if len(y) == 0 or (self.max_depth is not None and depth >= self.max_depth) or len(np.unique(y)) == 1:
            node.value = self._most_common_label(y)
            return node

        # Select random subset of features
        if self.max_features is not None:
            n_features_to_sample = min(self.max_features, n_features)
            feature_indices = np.random.choice(n_features, n_features_to_sample, replace=False)
        else:
            feature_indices = range(n_features)

        # Find best split from selected features
        best_gain = -1
        best_feature = None
        best_threshold = None

        for feature in feature_indices:
            thresholds = np.unique(X[:, feature])
            for threshold in thresholds:
                gain = self._information_gain(y, X[:, feature], threshold)
                if gain > best_gain:
                    best_gain = gain
                    best_feature = feature
                    best_threshold = threshold

        if best_gain == -1:
            node.value = self._most_common_label(y)
            return node

        # Create child nodes
        left_mask = X[:, best_feature] <= best_threshold
        right_mask = ~left_mask

        # Don't split if it creates empty nodes
        if np.sum(left_mask) == 0 or np.sum(right_mask) == 0:
            node.value = self._most_common_label(y)
            return node

        node.feature_index = best_feature
        node.threshold = best_threshold
        node.left = self._grow_tree(X[left_mask], y[left_mask], depth + 1)
        node.right = self._grow_tree(X[right_mask], y[right_mask], depth + 1)

        return node

    def _information_gain(self, y, feature, threshold):
        parent_entropy = self._entropy(y)

        left_mask = feature <= threshold
        right_mask = ~left_mask

        if len(y[left_mask]) == 0 or len(y[right_mask]) == 0:
            return 0

        n = len(y)
        n_l, n_r = len(y[left_mask]), len(y[right_mask])
        e_l, e_r = self._entropy(y[left_mask]), self._entropy(y[right_mask])
        child_entropy = (n_l/n) * e_l + (n_r/n) * e_r

        return parent_entropy - child_entropy

    def _entropy(self, y):
        if len(y) == 0:
            return 0
        hist = np.bincount(y)
        ps = hist / len(y)
        return -np.sum([p * np.log(p) for p in ps if p > 0])

    def _most_common_label(self, y):
        if len(y) == 0:
            return 0
        return Counter(y).most_common(1)[0][0]

    def predict(self, X):
        return np.array([self._traverse_tree(x, self.root) for x in X])

    def _traverse_tree(self, x, node):
        if node.value is not None:
            return node.value

        if x[node.feature_index] <= node.threshold:
            return self._traverse_tree(x, node.left)
        return self._traverse_tree(x, node.right)

class CustomRandomForest:
    def __init__(self, n_trees=10, max_depth=None, max_features=None):
        self.n_trees = n_trees
        self.max_depth = max_depth
        self.max_features = max_features
        self.trees = []

    def fit(self, X, y):
        self.trees = []

        # Default: sqrt(n_features) for classification
        if self.max_features is None:
            self.max_features = int(np.sqrt(X.shape[1]))

        for _ in range(self.n_trees):
            # Bootstrap sampling
            indices = np.random.choice(len(X), size=len(X), replace=True)
            sample_X = X[indices]
            sample_y = y[indices]

            # Train tree with random feature subset
            tree = CustomDecisionTree(max_depth=self.max_depth, max_features=self.max_features)
            tree.fit(sample_X, sample_y)
            self.trees.append(tree)

    def predict(self, X):
        # Get predictions from all trees
        tree_predictions = np.array([tree.predict(X) for tree in self.trees])
        # Majority vote
        return np.array([Counter(predictions).most_common(1)[0][0]
                        for predictions in tree_predictions.T])

In [10]:
# Load and prepare data
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/letter-recognition/letter-recognition.data"
column_names = ['lettr'] + [f'x{i}' for i in range(1, 17)]
data = pd.read_csv(url, names=column_names)

print("Dataset shape:", data.shape)

# Convert letters to numbers
label_map = {letter: i for i, letter in enumerate(data['lettr'].unique())}
y = data['lettr'].map(label_map).values
X = data.drop('lettr', axis=1).values

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train custom random forest
print("\nTraining Custom Random Forest...")
custom_rf = CustomRandomForest(n_trees=25, max_depth=12)
custom_rf.fit(X_train, y_train)
custom_predictions = custom_rf.predict(X_test)

accuracy = np.mean(custom_predictions == y_test)
print(f"\nCustom Random Forest Accuracy: {accuracy:.4f}")

Dataset shape: (20000, 17)

Training Custom Random Forest...

Custom Random Forest Accuracy: 0.9355
